<a href="https://colab.research.google.com/github/nicolaiberk/llm_ws/blob/main/notebooks/06a_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Informed Prompting

THIS NOTEBOOK IS IN DEVELOPMENT. CURRENTLY CANNOT INSTALL FAISS, FINALIZE NOTEBOOK AT A LATER POINT

> ❗ ACTIVATE THE GPU BY SELECTING RUNTIME IN THE UPPER RIGHT > CONNECT TO RUNTIME > T4 GPU

In [2]:
!pip install sentence_transformers datasets faiss-gpu-cu12 transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 119.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you

> ❗ RESTART THE NOTEBOOK (DROPDOWN NEXT TO RUN ALL > RESTART SESSION)

The [sentence-transformers](https://sbert.net/) library provides an ecosystem of models designed specifically for efficient embedding generation. It works very similar to transformers:

In [1]:
from sentence_transformers import SentenceTransformer, CrossEncoder

We load a pretrained model:

In [2]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Then we encode some sentences of interest:

In [3]:
sentences = [
    "The Great Wall of China was built over several dynasties, with most of the existing structure dating from the Ming Dynasty (1368-1644).",
    "The blue whale's heart alone can weigh as much as an automobile and is roughly the size of a small car.",
    "Studies show that the Dunning-Kruger effect causes people with low ability in a domain to overestimate their competence in that area.",
]

And encode them as embeddings:

In [4]:
# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)

(3, 384)


We can then calculate the cosine similarity of the sentences with each other:

In [5]:
# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[ 1.0000, -0.0797, -0.0810],
        [-0.0797,  1.0000,  0.0047],
        [-0.0810,  0.0047,  1.0000]])


## Similarity Search

This is particularly useful if we are searching something using a query:

In [6]:
query = "How large is a blue whales heart?"
query_embedding = model.encode([query])
similarities = model.similarity(query_embedding, embeddings)
print(similarities)

tensor([[ 0.0311,  0.6708, -0.0386]])


Looks good! Now we can then select the most similar context to add to the prompt:

In [7]:
best_index = similarities.squeeze().argmax().item() # get the index of the highest similarity

In [8]:
prompt = "Answer the Question. \nQuery:" + query + "\nContext: " + sentences[best_index]
print(prompt)

Answer the Question. 
Query:How large is a blue whales heart?
Context: The blue whale's heart alone can weigh as much as an automobile and is roughly the size of a small car.


We can try and

In [9]:
from datasets import load_dataset

dataset = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")

README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

data/passages.parquet/part.0.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/3200 [00:00<?, ? examples/s]

In [13]:
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Check for GPU availability and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Always clean + use this corpus consistently
corpus = []
for item in dataset["passages"]:
    text = str(item).strip()
    if text:
        corpus.append(text)

# Embedding model
print("Encoding corpus...")
embedder = SentenceTransformer("all-MiniLM-L6-v2").to(device)
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True).to(device)
corpus_embeddings_np = corpus_embeddings.cpu().numpy()

# FAISS index
index = faiss.IndexFlatL2(corpus_embeddings_np.shape[1])
index.add(corpus_embeddings_np)

# Reranker model (if uncommented)
# reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2").to(device)

# Generator (choose one: local HF model or OpenAI)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct", torch_dtype=torch.float16).to(device)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150, device=device)

# Embed query
query_embedding = embedder.encode([query], convert_to_tensor=True).to(device).cpu().numpy()

# Retrieve top-k from FAISS
D, I = index.search(query_embedding, k=5)
retrieved_docs = [corpus[idx] for idx in I[0]]

print("Retrieved indices:", I[0])
print("Retrieved docs:")
for doc in retrieved_docs:
    print("-", repr(doc))

Using device: cuda
Encoding corpus...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda


Retrieved indices: [1240 2594 3181 2590  953]
Retrieved docs:
- '{\'passage\': "With a mass just over 5 kg (11 lb), elephant brains are larger than those of any land animal, and although the largest whales have body masses twentyfold those of a typical elephant, whale brains are barely twice the mass of an elephant\'s. A wide variety of behaviour, including those associated with grief, making music, art, altruism, allomothering, play, use of tools,     compassion and self-awareness      evidence a highly intelligent species on par with cetaceans    and primates   .", \'id\': 1241}'
- "{'passage': 'Polar bears rank with the Kodiak bear as among the largest living land carnivores, and male polar bears may weigh twice as much as a Siberian tiger. Most adult males weigh 350 650 kg (770 1500+ lb) and measure 2.5 3.0 m (8.2 9.8 ft) in length. Adult females are roughly half the size of males and normally weigh 150 250 kg (330 550 lb), measuring 2 2.5 m (6.6 8.2 ft), but double their weight du

In [ ]:

# # Rerank
# rerank_pairs = [[str(query), str(doc)] for doc in retrieved_docs]
# scores = reranker.predict(rerank_pairs)
# reranked_docs = [doc for _, doc in sorted(zip(scores, retrieved_docs), reverse=True)]

In [15]:
# Combine for context
context = "\n\n".join(retrieved_docs[:2])
prompt = f"""Answer the following question using the provided context.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"""

# Generate
response = generator(prompt)[0]["generated_text"]

''

In [16]:
response

'Answer the following question using the provided context.\n\nContext:\n{\'passage\': "With a mass just over 5 kg (11 lb), elephant brains are larger than those of any land animal, and although the largest whales have body masses twentyfold those of a typical elephant, whale brains are barely twice the mass of an elephant\'s. A wide variety of behaviour, including those associated with grief, making music, art, altruism, allomothering, play, use of tools,     compassion and self-awareness      evidence a highly intelligent species on par with cetaceans    and primates   .", \'id\': 1241}\n\n{\'passage\': \'Polar bears rank with the Kodiak bear as among the largest living land carnivores, and male polar bears may weigh twice as much as a Siberian tiger. Most adult males weigh 350 650 kg (770 1500+ lb) and measure 2.5 3.0 m (8.2 9.8 ft) in length. Adult females are roughly half the size of males and normally weigh 150 250 kg (330 550 lb), measuring 2 2.5 m (6.6 8.2 ft), but double their 

In [18]:
generator(prompt)

[{'generated_text': 'Answer the following question using the provided context.\n\nContext:\n{\'passage\': "With a mass just over 5 kg (11 lb), elephant brains are larger than those of any land animal, and although the largest whales have body masses twentyfold those of a typical elephant, whale brains are barely twice the mass of an elephant\'s. A wide variety of behaviour, including those associated with grief, making music, art, altruism, allomothering, play, use of tools,     compassion and self-awareness      evidence a highly intelligent species on par with cetaceans    and primates   .", \'id\': 1241}\n\n{\'passage\': \'Polar bears rank with the Kodiak bear as among the largest living land carnivores, and male polar bears may weigh twice as much as a Siberian tiger. Most adult males weigh 350 650 kg (770 1500+ lb) and measure 2.5 3.0 m (8.2 9.8 ft) in length. Adult females are roughly half the size of males and normally weigh 150 250 kg (330 550 lb), measuring 2 2.5 m (6.6 8.2 ft

In [17]:
response.split("Answer:")

['Answer the following question using the provided context.\n\nContext:\n{\'passage\': "With a mass just over 5 kg (11 lb), elephant brains are larger than those of any land animal, and although the largest whales have body masses twentyfold those of a typical elephant, whale brains are barely twice the mass of an elephant\'s. A wide variety of behaviour, including those associated with grief, making music, art, altruism, allomothering, play, use of tools,     compassion and self-awareness      evidence a highly intelligent species on par with cetaceans    and primates   .", \'id\': 1241}\n\n{\'passage\': \'Polar bears rank with the Kodiak bear as among the largest living land carnivores, and male polar bears may weigh twice as much as a Siberian tiger. Most adult males weigh 350 650 kg (770 1500+ lb) and measure 2.5 3.0 m (8.2 9.8 ft) in length. Adult females are roughly half the size of males and normally weigh 150 250 kg (330 550 lb), measuring 2 2.5 m (6.6 8.2 ft), but double their